In [2]:
from collections import defaultdict
from datetime import datetime
import torch
import pandas as pd


In [3]:
m1_1m_path = 'datasets/ml-1m'

In [5]:
file_path = m1_1m_path + '/movies.dat'
data_movies = pd.read_csv(file_path, encoding='latin-1', sep='::', engine='python',
                   names=['MovieID', 'Title', 'Genres'], index_col='MovieID')
genres = data_movies['Genres'].str.get_dummies('|')
data_movies= pd.concat([data_movies, genres], axis=1)
data_movies.drop('Genres', axis=1, inplace=True)
data_movies

,Title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
MovieID,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Jumanji (1995),0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,Grumpier Old Men (1995),0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
4,Waiting to Exhale (1995),0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,Father of the Bride Part II (1995),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,Meet the Parents (2000),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3949,Requiem for a Dream (2000),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3950,Tigerland (2000),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [6]:
file_path = m1_1m_path + '/users.dat'
data_users = pd.read_csv(file_path, sep='::', engine='python',
                   names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], index_col='UserID')


# Convert Gender to binary indicator variable
data_users['Gender'] = (data_users['Gender'] == 'M').astype(int)
occupation_map = {
    0: 'other',
    1: 'academic/educator',
    2: 'artist',
    3: 'clerical/admin',
    4: 'college/grad student',
    5: 'customer service',
    6: 'doctor/health care',
    7: 'executive/managerial',
    8: 'farmer',
    9: 'homemaker',
    10: 'K-12 student',
    11: 'lawyer',
    12: 'programmer',
    13: 'retired',
    14: 'sales/marketing',
    15: 'scientist',
    16: 'self-employed',
    17: 'technician/engineer',
    18: 'tradesman/craftsman',
    19: 'unemployed',
    20: 'writer'
}
# data['Occupation'] = data['Occupation'].map(occupation_map)
data_users

,Gender,Age,Occupation,Zip-code
UserID,,,,
1,0,1,10,48067
2,1,56,16,70072
3,1,25,15,55117
4,1,45,7,02460
5,1,25,20,55455
...,...,...,...,...
6036,0,25,15,32603
6037,0,45,1,76006
6038,0,56,1,14706


In [60]:
file_path = m1_1m_path + '/ratings.dat'

data_ratings = pd.read_csv(file_path, sep='::', engine='python',
                   names=['UserID', 'MovieID', 'Rating', 'Timestamp'], index_col='UserID')


,MovieID,Rating,Timestamp
UserID,,,
1,1193,5,978300760
1,661,3,978302109
1,914,3,978301968
1,3408,4,978300275
1,2355,5,978824291
...,...,...,...
6040,1091,1,956716541
6040,1094,5,956704887
6040,562,5,956704746


In [8]:
data_ratings.loc[2]

,MovieID,Rating,Timestamp
UserID,,,
2,1357,5,978298709
2,3068,4,978299000
2,1537,4,978299620
2,647,3,978299351
2,2194,4,978299297
...,...,...,...
2,356,5,978299686
2,1245,2,978299200
2,1246,5,978299418


## SASRec handling
Only use implicit feedback in the sequence of items

In [19]:
data_ratings

,MovieID,Rating,Timestamp
UserID,,,
1,1193,5,978300760
1,661,3,978302109
1,914,3,978301968
1,3408,4,978300275
1,2355,5,978824291
...,...,...,...
6040,1091,1,956716541
6040,1094,5,956704887
6040,562,5,956704746


In [79]:

# use timestamps to determine the sequence order of actions.
order_ratings = data_ratings
order_ratings = order_ratings.groupby('UserID').filter(lambda  x: len(x) >= 5)
order_ratings = order_ratings.groupby('MovieID').filter(lambda  x: len(x) >= 5)
order_ratings = order_ratings.groupby('UserID').apply(lambda  x: x.sort_values('Timestamp'))
# We discard users and items with fewer than 5 related actions.
order_ratings

,MovieID,Rating,Timestamp
UserID,,,
1,3186,4,978300019
1,1721,4,978300055
1,1022,5,978300055
1,1270,5,978300055
1,2340,3,978300103
...,...,...,...
6040,2917,4,997454429
6040,1784,3,997454464
6040,1921,4,997454464


In [80]:
usermap = dict()
usernum = 0
itemmap = dict()
itemnum = 0
for _id, row in order_ratings.iterrows():
    if _id in usermap:
        userid = usermap[_id]
    else:
        usernum += 1
        userid = usernum
        usermap[_id] = userid

    if row.MovieID in itemmap:
        itemid = itemmap[row.MovieID]
    else:
        itemnum += 1
        itemid = itemnum
        itemmap[row.MovieID] = itemid

In [83]:
usermap.__len__(), itemmap.__len__()

(6040, 3416)

In [81]:
order_ratings['MovieID'] = order_ratings['MovieID'].map(itemmap)
order_ratings.index = order_ratings.index.map(usermap)
# data['Occupation'] = data['Occupation'].map(occupation_map)

In [82]:
order_ratings

,MovieID,Rating,Timestamp
UserID,,,
1,1,4,978300019
1,2,4,978300055
1,3,5,978300055
1,4,5,978300055
1,5,3,978300103
...,...,...,...
6040,1249,4,997454429
6040,88,3,997454464
6040,371,4,997454464


In [84]:
sas_data = order_ratings.drop('Rating', axis=1, inplace=False).drop('Timestamp', axis=1, inplace=False)
## ..continue
sas_data.loc[1]

,MovieID
UserID,
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9


In [85]:
with open('datasets/processed_SASRec/m1_1m.txt', 'w') as f:
    for _id in set(sas_data.index):
        for movie_id in sas_data.loc[_id].MovieID:
            f.write('%d %d\n' % (_id, movie_id))